In [4]:
import pandas as pd
from time import sleep
import json
import pprint

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
supplements = pd.read_csv("Supplements.csv")

In [8]:

supplements.head(5)

,Name,Url
0,Abscess Root,https://www.webmd.com/vitamins/ai/ingredientmo...
1,Abuta,https://www.webmd.com/vitamins/ai/ingredientmo...
2,Acacia Rigidula,https://www.webmd.com/vitamins/ai/ingredientmo...
3,Acai,https://www.webmd.com/vitamins/ai/ingredientmo...
4,Acerola,https://www.webmd.com/vitamins/ai/ingredientmo...


In [9]:
def get_overview(waitTime, supplementName):
    overview = ""
    try:        
        container = waitTime.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#overview-container .vitamins-monograph-content.overview-content')))
        div_text = container.text

        overview = div_text

    except Exception as e:
        print(f"Error in getting uses for: {supplementName} {e}")    
    finally:
        return overview

In [10]:
def get_uses(waitTime, supplementName):
    uses = {}
    try:
        container = waitTime.until(EC.presence_of_element_located((By.ID, "uses-container")))
        
        try:
            no_data = container.find_element(By.CSS_SELECTOR, "p.no-data-text")
            if no_data:
                return uses
        except:
            pass  
        
        parent_div = container.find_element(By.XPATH, "./div")
        child_divs = parent_div.find_elements(By.CSS_SELECTOR, ".vitamins-monograph-content.uses-content")
        
        tag_h3 = parent_div.find_elements(By.TAG_NAME, 'h3')
        
        if len(tag_h3) >= 1:
            ctr = 0  
            for child in child_divs:
                li_list = []
                
                ul_elements = child.find_elements(By.TAG_NAME, 'ul')
                
                if not ul_elements:
                    continue
                
                for ul in ul_elements:
                    li_elements = ul.find_elements(By.TAG_NAME, 'li')
                    for li in li_elements:
                        li_list.append([li.text])
                        
                uses[tag_h3[ctr].text] = li_list
                ctr += 1
                
    except Exception as e:
        print(f"Error in getting uses for: {supplementName} {e}")    
    finally:
        return uses


In [ ]:
def get_sideEffects(waitTime, supplementName):
    

In [ ]:
supplement_info = {}

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
waitTime = WebDriverWait(driver,10)

for _, info in supplements.iterrows():

    supplementName = info.Name
    supplementURL = info.Url
    
    driver.get(supplementURL)       

    overview = get_overview(waitTime, supplementName)
    uses = get_uses(waitTime, supplementName)
    #overview_content = parent_monograph.find_elements(By.CLASS_NAME, "vitamins-monograph-content overview-content")
    #text = overview_content.text.strip()
    
    
    #overview = get_overview_content()
    #overview = get_element_text(supplementName, "Overview", "/html/body/div[1]/div/main/div/div[2]/div/div/div[1]/div[1]/div")
    #uses = get_uses("/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]")
    #sideEffects = get_element_text(supplementName, "Side Effects", "/html/body/div[1]/div/main/div/div[2]/div/div/div[3]/div[1]")
    #precautions = get_element_text(supplementName, "Precautions", "/html/body/div[1]/div/main/div/div[2]/div/div/div[4]/div[1]/div")
    #interactons = [] /html/body/div[1]/div/main/div/div[2]/div/div/div[4]/div[2]/div
    #dosing = get_element_text(supplementName, "Dosing", "/html/body/div[1]/div/main/div/div[2]/div/div/div[6]/div[1]/div")

    supplement_info[supplementName] = {
    #     "Overview": overview,
    #     "Uses": uses,
    #     "SideEffects": [],
    #     "Precautions": [],
    #     "Interactions": [],
    #     "Dosing": [] 
    }
    sleep(1)
driver.quit()

Error in getting uses for: Aloe Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="uses-container"]"}
  (Session info: chrome=134.0.6998.167); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00CEC7F3+24435]
	(No symbol) [0x00C72074]
	(No symbol) [0x00B406E3]
	(No symbol) [0x00B88B39]
	(No symbol) [0x00B88E8B]
	(No symbol) [0x00BD1AC2]
	(No symbol) [0x00BAD804]
	(No symbol) [0x00BCF20A]
	(No symbol) [0x00BAD5B6]
	(No symbol) [0x00B7C54F]
	(No symbol) [0x00B7D894]
	GetHandleVerifier [0x00FF70A3+3213347]
	GetHandleVerifier [0x0100B0C9+3295305]
	GetHandleVerifier [0x0100558C+3271948]
	GetHandleVerifier [0x00D87360+658144]
	(No symbol) [0x00C7B27D]
	(No symbol) [0x00C78208]
	(No symbol) [0x00C783A9]
	(No symbol) [0x00C6AAC0]
	BaseThreadInitThunk [0x762B7BA9+25]
	RtlInitializeExceptionChain [0x7738C2EB+107]
	RtlClearBits

In [94]:
supplement_info

{'Abscess Root': {'Uses': {}},
 'Abuta': {'Uses': {}},
 'Acacia Rigidula': {'Uses': {}},
 'Acai': {'Uses': {}},
 'Acerola': {'Uses': {'Possibly Effective for': [['Vitamin C deficiency. Acerola is a rich source of vitamin C. Taking acerola by mouth might help prevent or treat vitamin C deficiency.']]}},
 'Acetyl-L-Carnitine': {'Uses': {'Possibly Effective for': [['Decline in memory and thinking skills that occurs normally with age. Taking acetyl-L-carnitine by mouth might improve memory and mental function in older people with some memory loss.'],
    ['Tiredness in older adults. Taking acetyl-L-carnitine by mouth might improve mental and physical tiredness in older people. It also seems to reduce tiredness after exercise.'],
    ["Alcohol use disorder. Acetyl-L-carnitine given intravenously (by IV) for 10 days and then taken by mouth for 80 days, helps reduce withdrawal symptoms and cravings during alcohol detox. But it's possible that the benefits are from IV acetyl-L-carnitine only."

In [95]:
pprint.pprint(supplement_info)

{'Abscess Root': {'Uses': {}},
 'Abuta': {'Uses': {}},
 'Acacia Rigidula': {'Uses': {}},
 'Acai': {'Uses': {}},
 'Acerola': {'Uses': {'Possibly Effective for': [['Vitamin C deficiency. '
                                                  'Acerola is a rich source of '
                                                  'vitamin C. Taking acerola '
                                                  'by mouth might help prevent '
                                                  'or treat vitamin C '
                                                  'deficiency.']]}},
 'Acetyl-L-Carnitine': {'Uses': {'Possibly Effective for': [['Decline in '
                                                             'memory and '
                                                             'thinking skills '
                                                             'that occurs '
                                                             'normally with '
                                               

In [136]:
print(json.dumps(supplement_info, indent=4))

{
    "Abscess Root": {
        "Overview": "Abscess root (Polemonium reptans) is an herb. The root is ground and used as a medicinal tea, but it's not clear if it has any benefits.\n\nAbscess root is used for fever, cough, swelling, and other conditions. But there is no good scientific evidence to support any uses.\n\nAbscess root is also called \"false Jacob's ladder,\" because it has some of the same effects as another plant called Jacob's ladder. These two plants are used in similar ways, but are not the same.",
        "Uses": [],
        "SideEffects": [],
        "Precautions": [],
        "Interactions": [],
        "Dosing": []
    }
}


TEST HERE

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.webmd.com/vitamins/ai/ingredientmono-266/abscess-root")
wait_time = WebDriverWait(driver,5)
#div = "/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]"
/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]/div[3]
div = "/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/p"
elem = wait_time.until(EC.presence_of_element_located((By.XPATH, div)))
elem = wait_time.until(EC.)
print(elem.text)
driver.quit()